In [ ]:
from aim2dat.aiida_workflows.workflow_builder import WorkflowBuilder
import aiida

aiida.load_profile("tests")
wf_builder = WorkflowBuilder()

## The workflow protocol

The workflow protocols consists of three different sections:

* _tasks_: Is a dictionary containing the details and dependencies for the tasks that can be run with the current workflow.
* _general_input_: defines the preset parameters shared by all work chains.
* _user_input_: defines input parameters that are set by the user.

All predefined protocols are found in the folder: "aim2dat/aim2dat/aiida_workflows/protocols/".
The workflow protocols support versions, which the suffix `"_v*.*"` (* denotes an integer number) a specific protocol version can be chosen.
If the suffix is omitted the latest protocol version is chosen.
At the moment the following protocols are implemented:

| Protocol                            | Latest version | Description                    |
| ----------------------------------- | -------------- | ------------------------------ |
| _arithmetic-testing_                | v1.1           | Protocol for testing purposes. |
| _seekpath-standard_                 | v1.0           | Protocol for a seek-path analysis. |
| _cp2k-crystal-mofs_                 | v2.0           | Protocol to run DFT calculations on MOFs using CP2K. |
| _cp2k-crystal-preopt_               | v3.1           | Protocol to pre-optimize inorganic crystals with loose parameters using CP2K. |
| _cp2k-crystal-standard_             | v3.2           | Standard protocol to run DFT calculations on inorganic crystals using CP2K (<a href="https://doi.org/10.1063/5.0082710" target="_blank">doi:10.1063/5.0082710</a>). |
| _cp2k-crystal-standard-keep-angles_ | v1.1           | Standard protocol for inorganic crystals but constraining lattice parameters. |
| _cp2k-surface-standard_             | v1.0           | Protocol to run the surface workflow using CP2K. |
| _cp2k-crystal-testing_              | v2.0           | Protocol to test the workflow for inorganic crystals with loose parameters using CP2K. |
| _cp2k-surface-testing_              | v1.0           | Protocol to test the surface workflow with loose parameters using CP2K. |

In [ ]:
wf_builder.protocol = "arithmetic-testing"

In [ ]:
wf_builder.tasks

## Setting up the input parameters and parent node

The provenance of the workflow is defined via the parent node, it is input for all initial tasks of the workflow.
Here, we create a new aiida node without history and pass it to the builder-object:

In [ ]:
from aiida.plugins import DataFactory

Float = DataFactory("core.float")
parent_node = Float(4)

wf_builder.parent_node = parent_node

And we can set additional input-parameters (parameters can be given as python types or AiiDA nodes). A dash and subsequent greater than sign (_->_) highlight an individual input parameter defined for just one task of the workflow.

In [ ]:
wf_builder.set_user_input("y", 5)
wf_builder.set_user_input("y->task_4.1", 11.0)

In [ ]:
wf_builder.determine_workflow_state()

The builder checks whether any work chains with matching input parameters have been performed on the structure. In this case there are no processes run that conform the workflow protocol.

## Executing workflow tasks

The input for the initial task can be created using the 'builder'-method of the AiiDA work chain or calculation or a dictionary of input-parameters for AiiDA calcfunctions:

In [ ]:
from aiida.engine import run

wc_builder = wf_builder.generate_inputs("task_1.1")
result = run(**wc_builder)

If we check the workflow again, we see that the task 'task_1.1' is accomplished and we can continue with the next task:

In [ ]:
wf_builder.determine_workflow_state()

Alternatively, we can run or submit the task straightaway using the methods :meth:`run_task <aim2dat.aiida_workflows.workflow_builder.WorkflowBuilder.run_task>` or :meth:`submit_task <aim2dat.aiida_workflows.workflow_builder.WorkflowBuilder.submit_task>`.
The difference between the two methods is that the first uses AiiDA's `run` method which starts the process in the foreground and blocks the interface while the latter uses AiiDA's `submit` method which passes the process to the daemon that is running in the background.

In [ ]:
wf_builder.run_task("task_1.2")
wf_builder.run_task("task_1.3")
wf_builder.run_task("task_2.1")

## Visualizing the provenance graph of the workflow

Using the AiiDA built-in features the provenance graph of the workflow can be plotted:

In [ ]:
wf_builder.graph_attributes = {"graph_attr": {"size": "6!,6"}}
graph = wf_builder.generate_provenance_graph()
graph.graphviz

In [ ]:
from aim2dat.aiida_workflows.workflow_builder import MultipleWorkflowBuilder

mwf_builder = MultipleWorkflowBuilder()
mwf_builder.protocol = "arithmetic-testing"

for n in range(0, 5):
    mwf_builder.add_parent_node(Float(n))

In [ ]:
mwf_builder.set_user_input("y", 2.0)
mwf_builder.set_user_input("y->task_4.1", 3.0)

The status information as well as process nodes and workflow results is therefore given as pandas dataframes:

In [ ]:
mwf_builder.return_workflow_states()

Different tasks can be started for all `parent-nodes` within one function call via the :meth:`run_task <aim2dat.aiida_workflows.workflow_builder.MultipleWorkflowBuilder.run_task>` or :meth:`submit_task <aim2dat.aiida_workflows.workflow_builder.MultipleWorkflowBuilder.submit_task>` functions:

In [ ]:
mwf_builder.run_task("task_1.1")
mwf_builder.return_workflow_states()

The tasks can be started for a subset of the _parent-nodes_ by using the ``interval`` parameter:

In [ ]:
mwf_builder.run_task("task_1.2", interval=[0, 3])
mwf_builder.return_workflow_states()

Several tasks can be started consecutively by setting a task queue:

In [ ]:
mwf_builder.add_to_task_queue("task_1.2", run_type="run")
mwf_builder.add_to_task_queue("task_1.3", run_type="run")
mwf_builder.add_to_task_queue("task_2.1", run_type="run")
mwf_builder.add_to_task_queue("task_2.2", run_type="run")
mwf_builder.add_to_task_queue("task_3.1", run_type="run")
mwf_builder.add_to_task_queue("task_4.1", run_type="run")
mwf_builder.execute_task_queue()

Additional information can be returned via the functions [``return_process_nodes``](aim2dat.aiida_workflows.workflow_builder.MultipleWorkflowBuilder.return_process_nodes) and [``return_results``](aim2dat.aiida_workflows.workflow_builder.MultipleWorkflowBuilder.return_results):

In [ ]:
mwf_builder.return_process_nodes()

In [ ]:
mwf_builder.return_results()

In [ ]:
mwf_builder.to_file("test_workflow.yaml")

mwf_builder2 = MultipleWorkflowBuilder.from_file("test_workflow.yaml")
mwf_builder2.return_workflow_states()